# Best Hybrid Search Configuration

This notebook runs different hybrid search configurations, calculates the metrics for each configuration and compares the results to the metrics calcuated after the baseline run from the previous notebook. 

We are using the same query set to have a fair comparison.

## Get queries

In [59]:
import pandas as pd
import requests
import pytrec_eval
import json
import mercury as mr
from IPython.display import display, HTML, Image


app = mr.App(title="Let's Run a Hybrid Search", static_notebook=True)

mercury.App

In [60]:
name = 'queries.txt'
#df_query_idx = pd.read_csv(name, sep="\t", names=['query'], index_col=0)
df_query_idx = pd.read_csv(name, sep="\t", names=['idx', 'query'])

In [61]:
df_query_idx

,idx,query
0,0,$30 roblox gift card not digital
1,1,(fiction without frontiers)
2,2,100
3,3,10x10x6 cake box without window
4,4,15 inch light weight laptop that has lots of m...
...,...,...
215,215,wooden stool
216,216,woodwick wax melt
217,217,world of warcraft anniversary collector's edition
218,218,wowled


In [62]:
name = 'ratings.qrels'

df_ratings = pd.read_csv(name, sep="\t", names=['idx', 'Q0', 'docid', 'rating'])#, index=False)
df_ratings

,idx,Q0,docid,rating
0,0,0,B07RX6FBFR,3
1,0,0,B09194H44R,0
2,0,0,B08R5N6W6B,2
3,0,0,B07Y693ND1,0
4,0,0,B07RZ75JW3,2
...,...,...,...,...
4060,219,0,B00JX10Q2O,2
4061,219,0,B00KY41UHO,3
4062,219,0,B00QXJOUL2,3
4063,219,0,B00UY14WCM,3


## Query OpenSearch with the Hybrid Search Configurations

In [63]:
keyword_weight = 0.3

In [64]:
neural_weight = round(1.0 - keyword_weight, 2)
print(f"Keyword Weight is {keyword_weight} and Neural Weight is {neural_weight}")

Keyword Weight is 0.3 and Neural Weight is 0.7


In [65]:
# Get model_id
url = "http://localhost:9200/_plugins/_ml/models/_search"

headers = {
    'Content-Type': 'application/json'
}

payload = {
  "query": {
    "match_all": {}
  },
  "size": 1
}

response = requests.request("POST", url, headers=headers, data=json.dumps(payload))

#hits.hits[0]['model_id']

model_id = response.json()['hits']['hits'][0]['_source']['model_id']

In [66]:
url = "http://localhost:9200/_search/pipeline/hybrid-search-pipeline"

print(f"Setting default model id to: {model_id}")
payload = {
  "request_processors": [
    {
      "neural_query_enricher" : {
        "description": "Sets the default model ID at index and field levels",
        "default_model_id": model_id,
        "neural_field_default_id": {
           "title_embeddings": model_id
        }
      }
    }
  ],
  "phase_results_processors": [
    {
      "normalization-processor": {
        "normalization": {
          "technique": "min_max"
        },
        "combination": {
          "technique": "arithmetic_mean",
          "parameters": {
            "weights": [
              keyword_weight,
              neural_weight
            ]
          }
        }
      }
    }
  ]    
}


response = requests.request("PUT", url, headers=headers, data=json.dumps(payload))
mr.JSON(response.json(), level=4)

Setting default model id to: zZrgMpIBFSlgWAuG9zO3


In [67]:
df_query_idx

,idx,query
0,0,$30 roblox gift card not digital
1,1,(fiction without frontiers)
2,2,100
3,3,10x10x6 cake box without window
4,4,15 inch light weight laptop that has lots of m...
...,...,...
215,215,wooden stool
216,216,woodwick wax melt
217,217,world of warcraft anniversary collector's edition
218,218,wowled


In [68]:
url = "http://localhost:9200/ecommerce/_search?search_pipeline=hybrid-search-pipeline"

headers = {
    'Content-Type': 'application/json'
}

df_relevance = pd.DataFrame()

for query in df_query_idx.itertuples():

    payload = {
      "_source": {
        "excludes": [
          "title_embedding"
        ]
      },
      "query": {
        "hybrid": {
          "queries": [
            {
              "match": {
                "title_text": {
                  "query": query[2]
                }
              }
            },
            {
              "neural": {
                "title_embedding": {
                  "query_text": query[2],
                  "k": 50
                }
              }
            }
          ]
        }
      }
    }

    response = requests.request("POST", url, headers=headers, data=json.dumps(payload)).json()
    #print(query[1])
    #print(query[2])
    #mr.JSON(response, level=0)
    
    position = 0
    #print(response['hits']['total'])
    for hit in response['hits']['hits']:
        # create a new row for the DataFrame and append it
        row = { 'query_id' : str(query[1]), 'query_string': query[2],  'Q0' : "Q0", 'product_id' : hit["_id"], 'position' : str(position), 'relevance' : hit["_score"], 'run': 'default' }
        #df_relevance = df_relevance.append(row, ignore_index=True)
        #df_relevance.loc[len(df_relevance)] = row

        new_row_df = pd.DataFrame([row])
        df_relevance = pd.concat([df_relevance, new_row_df], ignore_index=True)
        #print("%(id)s %(title)s: %(name)s" % hit["_source"])
        position += 1
    
# store the DataFrame without header and index, with tabs as delimiters
#name = '../data/default_result'
#df_relevance.to_csv(name, sep="\t", header=False, index=False)

# work with two for loops:
# 1) one to iterate over the list of queries and have a query id instead of a query
# 2) another one to iterate over the result sets to have the position of the result in the result set 

# DataFrame with columns:
# query_id: the id of the query as the trec_eval tool needs a numeric id rather than a query string as an identifier
# Q0: all lines have Q0, currently unused by trec_eval
# product_id: the id of the product in the hit list
# position: the position of the product in the result set
# relevance: relevance as given by the search engine
# run: the name of the query run

In [69]:
df_relevance

,query_id,query_string,Q0,product_id,position,relevance,run
0,0,$30 roblox gift card not digital,Q0,B00F4CF4PU,0,0.700000,default
1,0,$30 roblox gift card not digital,Q0,B07C438TMN,1,0.332253,default
2,0,$30 roblox gift card not digital,Q0,B00XJZHJCA,2,0.323889,default
3,0,$30 roblox gift card not digital,Q0,B00GAC1D2G,3,0.279684,default
4,0,$30 roblox gift card not digital,Q0,B004RMK4BC,4,0.227564,default
...,...,...,...,...,...,...,...
2195,219,yarn purple and pink,Q0,B07YYHG1KB,5,0.121923,default
2196,219,yarn purple and pink,Q0,B00J3YPKCW,6,0.106524,default
2197,219,yarn purple and pink,Q0,B07RNRTM38,7,0.018004,default
2198,219,yarn purple and pink,Q0,B0055734DS,8,0.010082,default


## Transform data to meet the `pytrec_eval` requirements

### Convert string ids to integer values

In [70]:
unique_ids = pd.Series(pd.concat([df_relevance['product_id'], df_ratings['docid']]).unique())

# Create a mapping of each unique identifier to an integer
id_to_int = {id_val: idx for idx, id_val in enumerate(unique_ids, start=1)}

# Map the identifiers in both DataFrames
df_relevance['product_id_int'] = df_relevance['product_id'].map(id_to_int)
df_ratings['docid_int'] = df_ratings['docid'].map(id_to_int)

In [71]:
df_relevance

,query_id,query_string,Q0,product_id,position,relevance,run,product_id_int
0,0,$30 roblox gift card not digital,Q0,B00F4CF4PU,0,0.700000,default,1
1,0,$30 roblox gift card not digital,Q0,B07C438TMN,1,0.332253,default,2
2,0,$30 roblox gift card not digital,Q0,B00XJZHJCA,2,0.323889,default,3
3,0,$30 roblox gift card not digital,Q0,B00GAC1D2G,3,0.279684,default,4
4,0,$30 roblox gift card not digital,Q0,B004RMK4BC,4,0.227564,default,5
...,...,...,...,...,...,...,...,...
2195,219,yarn purple and pink,Q0,B07YYHG1KB,5,0.121923,default,2184
2196,219,yarn purple and pink,Q0,B00J3YPKCW,6,0.106524,default,2185
2197,219,yarn purple and pink,Q0,B07RNRTM38,7,0.018004,default,2186
2198,219,yarn purple and pink,Q0,B0055734DS,8,0.010082,default,2187


In [72]:
df_ratings

,idx,Q0,docid,rating,docid_int
0,0,0,B07RX6FBFR,3,2189
1,0,0,B09194H44R,0,2190
2,0,0,B08R5N6W6B,2,2191
3,0,0,B07Y693ND1,0,2192
4,0,0,B07RZ75JW3,2,2193
...,...,...,...,...,...
4060,219,0,B00JX10Q2O,2,5611
4061,219,0,B00KY41UHO,3,5612
4062,219,0,B00QXJOUL2,3,2181
4063,219,0,B00UY14WCM,3,5613


In [73]:
# Drop the Q0 column as it is not needed
df_pytrec_qrels = df_ratings.drop(columns=['Q0', 'docid'])

df_pytrec_qrels['docid_int'] = df_pytrec_qrels['docid_int'].astype(str)

# Initialize an empty dictionary to store the final qrel structure
qrel = {}

# Group by 'idx' (which corresponds to 'q1', 'q2', etc.)
for idx, group in df_pytrec_qrels.groupby('idx'):
    # Create a dictionary for each group where 'docid' is the key and 'rating' is the value
    qrel[str(idx)] = dict(zip(group['docid_int'], group['rating']))
#print(qrel)

In [74]:
df_pytrec_results = df_relevance.drop(columns=['Q0', 'position', 'run', 'product_id'])

df_pytrec_results['relevance'] = df_pytrec_results['relevance'].astype(int)
df_pytrec_results['product_id_int'] = df_pytrec_results['product_id_int'].astype(str)

# Initialize an empty dictionary to store the final 'run' structure
run = {}

# Group by 'query_id' (which corresponds to 'q1', 'q2', etc.)
for query_id, group in df_pytrec_results.groupby('query_id'):
    # Create a dictionary for each group where 'product_id' is the key and 'relevance' is the value
    run[query_id] = dict(zip(group['product_id_int'], group['relevance']))

# Print the resulting run structure
#print(run)

In [75]:
evaluator = pytrec_eval.RelevanceEvaluator(
    qrel, {'map', 'ndcg'})

data = evaluator.evaluate(run)

print(json.dumps(data, indent=1))

{
 "0": {
  "map": 0.05555555555555555,
  "ndcg": 0.12135535928821184
 },
 "1": {
  "map": 0.0870748299319728,
  "ndcg": 0.24092784177141885
 },
 "10": {
  "map": 0.3522486772486772,
  "ndcg": 0.5800209401863611
 },
 "100": {
  "map": 0.2333333333333333,
  "ndcg": 0.32732846947743477
 },
 "101": {
  "map": 0.0,
  "ndcg": 0.0
 },
 "102": {
  "map": 0.0803030303030303,
  "ndcg": 0.2606048005042146
 },
 "103": {
  "map": 0.0,
  "ndcg": 0.0
 },
 "104": {
  "map": 0.014285714285714287,
  "ndcg": 0.07233057525796258
 },
 "105": {
  "map": 0.0125,
  "ndcg": 0.09382209734404523
 },
 "106": {
  "map": 0.02111801242236025,
  "ndcg": 0.08339507081886648
 },
 "107": {
  "map": 0.03125,
  "ndcg": 0.11111505071373692
 },
 "108": {
  "map": 0.625,
  "ndcg": 0.7287373994408333
 },
 "109": {
  "map": 0.05,
  "ndcg": 0.1570308682572845
 },
 "11": {
  "map": 0.13888888888888887,
  "ndcg": 0.3239555569460253
 },
 "110": {
  "map": 0.0,
  "ndcg": 0.0
 },
 "111": {
  "map": 0.040451388888888884,
  "ndcg": 0

In [76]:
ndcg_sum = 0
map_sum = 0
num_queries = len(data)

# Iterate over the dictionary and sum the 'ndcg' and 'map' values
for query, metrics in data.items():
    ndcg_sum += metrics['ndcg']
    map_sum += metrics['map']

# Calculate the averages
average_ndcg = ndcg_sum / num_queries
average_map = map_sum / num_queries

# Print the results
print("Baseline metrics")
print(f"Average NDCG: {average_ndcg}")
print(f"Average MAP: {average_map}")

Baseline metrics
Average NDCG: 0.18840541574327768
Average MAP: 0.1003479108726933
